In [14]:
!pip install sympy numpy pandas ht
import sympy as smp
import numpy as np
import pandas as pd
import ht

In [17]:
#Datos del enunciado
Temps_fluidos = (278,268,258,248,238,228)
esp_aislantes = (0.025,0.040,0.050,0.065,0.075,0.090,0.1,0.115,0.125)

data = np.empty((0,4))

#Medidas normalizadas de las cañerías
d_nom = (1, 1.25, 1.5, 2, 2.5, 3, 3.5, 4)
esp_caño = (0.00338, 0.00356, 0.00368, 0.00391, 0.00516, 0.00549, 0.00574, 0.00602) 
d_int = (0.02664, 0.03508, 0.04094, 0.05248, 0.06268, 0.07792, 0.09012, 0.10226)

diam = float(input("Ingrese (en pulgadas decimales) el diámetro nominal de cañería a analizar: "))
k = d_nom.index(diam)

#Temperatura de rocío
T_r = 36.9 + 273 #K
T_r_C = np.round(T_r-273,3)

print(f"La temperatura de rocío es: {T_r_C}°C")
print(f'Para un caño de diámetro nominal {d_nom[k]}": ')

p = 0 #Junto con la variable j se logra que al cambiar de T_f se comience a iterar con e_optimo ya que nunca será menor al aumentar el delta T. 

for i in Temps_fluidos:
    j = 0
    while j>=0:
        Q_tot = smp.symbols('Q_tot', real=True, positive=True)

        #Temperatura del fluido
        T_f = i #K

        #Espesor del aislante
        e_aisl = esp_aislantes[p] #m

        #Radio interior del caño (d/2)
        r_int = d_int[k]/2 #[m]

        #Espesor del caño
        e_caño = esp_caño[k] #[m] Se obtiene de la tabla provista para schedule 40"

        h_int = 850 #[W/m^2K]
        r_ext_caño = r_int + e_caño 
        k_acero = ht.k_material(ht.nearest_material('steel')) #[W/mK]
        r_ext_aisl = r_ext_caño + e_aisl
        k_aisl = 0.038 #[W/mK]Espuma elastomérica flexible (EE)
        r_ext = r_ext_aisl + 0.0005
        k_recub = 15 #[W/mK]
        emis_recub = 0.1
        sigma = 5.67e-8 #[W/m**2K**4]
        h_ext = 34 #[W/m^2K]

        #Temperatura exterior
        T1 = (38+273) #K

        #Resistencias térmicas (serie)
        R_conv_int = 1/(2*np.pi*r_int*h_int)
        R_cond_caño = smp.log(r_ext_caño/r_int)/(2*np.pi*k_acero)
        R_cond_aisl = smp.log(r_ext_aisl/r_ext_caño)/(2*np.pi*k_aisl)
        R_cond_recub = smp.log(r_ext/r_ext_aisl)/(2*np.pi*k_recub)
        R_eq_serie = R_conv_int + R_cond_caño + R_cond_aisl + R_cond_recub

        T_recub = T_f + (Q_tot * R_eq_serie) #Aplicamos ley de Ohm para la temperatura en el recubrimiento
        Tm = (T1+T_recub)/2

        #Resistencias térmicas (paralelo)
        R_rad_ext = 1/(4*emis_recub*sigma*Tm**3*2*np.pi*r_ext*(T1-T_recub))
        R_conv_ext = (1/(2*np.pi*r_ext*h_ext))
        R_eq_paralelo = (R_rad_ext*R_conv_ext)/(R_rad_ext+R_conv_ext)

        R_eq_tot = R_eq_serie + R_eq_paralelo

        Q_res = (T1-T_f)/R_eq_tot

        sol = smp.solve(Q_res - Q_tot, Q_tot) #Resolución de la ecuación no lineal
        calor_temp = np.array([sol[0],sol[0]*R_eq_serie + T_f])
        if calor_temp[0] >= 25 or calor_temp[1]<T_r: #Si no se cumple alguna condición se aumenta e_aisl
            p += 1
            continue
        data = np.append(data,[[T_f-273, e_aisl*1000, calor_temp[0], calor_temp[1]-273]], axis=0)
        resultado = f'Temperatura del fluido: {T_f-273} °C. Espesor óptimo: {(e_aisl*1000):.0f} mm. Flujo de calor: {calor_temp[0]:.2f} W/m. Temperatura en el recubrimiento = {(calor_temp[1]-273):.2f} °C'
        print(resultado) #Útil para ver el progreso de la corrida
        p = esp_aislantes.index(e_aisl)
        j = -1

Ingrese (en pulgadas decimales) el diámetro nominal de cañería a analizar: 4
La temperatura de rocío es: 36.9°C
Para un caño de diámetro nominal 4": 
Temperatura del fluido: 5 °C. Espesor óptimo: 40 mm. Flujo de calor: 14.51 W/m. Temperatura en el recubrimiento = 37.31 °C
Temperatura del fluido: -5 °C. Espesor óptimo: 40 mm. Flujo de calor: 18.91 W/m. Temperatura en el recubrimiento = 37.11 °C
Temperatura del fluido: -15 °C. Espesor óptimo: 40 mm. Flujo de calor: 23.32 W/m. Temperatura en el recubrimiento = 36.91 °C
Temperatura del fluido: -25 °C. Espesor óptimo: 50 mm. Flujo de calor: 23.51 W/m. Temperatura en el recubrimiento = 37.00 °C
Temperatura del fluido: -35 °C. Espesor óptimo: 65 mm. Flujo de calor: 22.65 W/m. Temperatura en el recubrimiento = 37.15 °C
Temperatura del fluido: -45 °C. Espesor óptimo: 75 mm. Flujo de calor: 23.38 W/m. Temperatura en el recubrimiento = 37.19 °C


In [18]:
df = pd.DataFrame(data, columns = ['Temperatura del fluido (°C)','Espesor de aislante óptimo (mm)', 'Flujo de calor (W/m)', 'Temperatura en el recubrimiento (°C)'])
styles = [dict(selector="caption", 
    props=[("text-align", "center"),
    ("font-size", "120%"),
    ("color", 'black')])]
df.style.format({
    'Flujo de calor (W/m)': '{:,.2f}'.format,
    'Temperatura en el recubrimiento (°C)': '{:,.2f}'.format,
    'Temperatura del fluido (°C)': '{:,.0f}'.format,
    'Espesor de aislante óptimo (mm)': '{:,.0f}'.format
    }).set_caption(f'Diámetro {diam}"').set_properties(**{'text-align': 'center'})\
    .set_table_styles(styles).hide(axis="index")

Temperatura del fluido (°C),Espesor de aislante óptimo (mm),Flujo de calor (W/m),Temperatura en el recubrimiento (°C)
5,40,14.51,37.31
-5,40,18.91,37.11
-15,40,23.32,36.91
-25,50,23.51,37.00
-35,65,22.65,37.15
-45,75,23.38,37.19
